In [1]:
import keras
import tensorflow as tf
from keras import layers
import make_val_prepare_data_aclImdb as imdb_data
import numpy as np


max_length = 600
max_tokens = 10000

path_to_glove_file = "/home/binbin/dl/python_deep_learning/glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs
print(f"Found {len(embeddings_index)} word vecotrs.")


2024-12-20 23:47:56.349446: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-20 23:47:56.357857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-20 23:47:56.371977: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-20 23:47:56.375066: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 23:47:56.383533: I tensorflow/core/platform/cpu_feature_guar

Found 22500 files belonging to 2 classes.


I0000 00:00:1734709678.266748   27442 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734709678.312582   27442 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734709678.312637   27442 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734709678.314457   27442 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734709678.314498   27442 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Found 2500 files belonging to 1 classes.
Found 25000 files belonging to 2 classes.
train_ds
input shape: (32,)
input shape: <dtype: 'string'>
targets shape: (32,)
targets shape: <dtype: 'int32'>
input[0]: (32,)
targets[0]: <dtype: 'int32'>
val_ds
input shape: (32,)
input shape: <dtype: 'string'>
targets shape: (32,)
targets shape: <dtype: 'int32'>
input[0]: (32,)
targets[0]: <dtype: 'int32'>
test_ds
input shape: (32,)
input shape: <dtype: 'string'>
targets shape: (32,)
targets shape: <dtype: 'int32'>
input[0]: (32,)
targets[0]: <dtype: 'int32'>
Found 400000 word vecotrs.


In [2]:
import make_val_prepare_data_aclImdb as imdb_data

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length
)

text_only_train_ds = imdb_data.train_ds.map(lambda x,y:x)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = imdb_data.train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = imdb_data.val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = imdb_data.test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

2024-12-20 23:48:11.084971: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [3]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim)) 
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

In [4]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 100) │  1,000,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None)      │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 64)        │     34,048 │ embedding[0][0],  │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1)         │         65 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,034,113 (3.94 MB)

 Trainable params: 34,113 (133.25 KB)

 Non-trainable params: 1,000,000 (3.81 MB)

In [5]:
callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True),
    keras.callbacks.TensorBoard(log_dir="glove_embeddings_sequence_model")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=50,
          callbacks=callbacks)


Epoch 1/50


2024-12-20 23:48:13.546897: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90100


704/704 ━━━━━━━━━━━━━━━━━━━━ 80s 111ms/step - accuracy: 0.6451 - loss: 0.6237 - val_accuracy: 0.7880 - val_loss: 0.4703
Epoch 2/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 89s 127ms/step - accuracy: 0.7858 - loss: 0.4640 - val_accuracy: 0.7324 - val_loss: 0.6469
Epoch 3/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 79s 113ms/step - accuracy: 0.8195 - loss: 0.4080 - val_accuracy: 0.7152 - val_loss: 0.5806
Epoch 4/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 61s 86ms/step - accuracy: 0.8455 - loss: 0.3695 - val_accuracy: 0.9048 - val_loss: 0.2518
Epoch 5/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 89s 127ms/step - accuracy: 0.8563 - loss: 0.3408 - val_accuracy: 0.7392 - val_loss: 0.6248
Epoch 6/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 90s 128ms/step - accuracy: 0.8663 - loss: 0.3219 - val_accuracy: 0.8504 - val_loss: 0.3592
Epoch 7/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 90s 128ms/step - accuracy: 0.8726 - loss: 0.3059 - val_accuracy: 0.7524 - val_loss: 0.5878
Epoch 8/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 90s 128ms/step - accuracy: 0.8780 - loss: 0.2917 - val_

In [6]:
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 44s 56ms/step - accuracy: 0.8688 - loss: 0.3186
Test acc: 0.868
